In [29]:
import pandas as pd
import numpy as np
import bokeh
import seaborn as sb
from scipy.stats import kurtosis, skew
from bokeh.core.properties import value
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.layouts import gridplot
from bokeh.models import PanTool, ResetTool, WheelZoomTool, HoverTool, LassoSelectTool, BoxSelectTool, ColumnDataSource, ranges, LabelSet

data = pd.read_csv('../Data/clean/finalclean3.0.csv', encoding = 'latin-1', low_memory = False)
data.dtypes

JAARTAL                    int64
STRAATNAAM                object
POSTCODE_VAN              object
POSTCODE_TOT              object
PRODUCTSOORT               int64
Aantal Aansluitingen       int64
%Leveringsrichting         int64
%Fysieke status            int64
%Soort aansluiting         int64
Soort aansluiting Naam    object
SJV                        int64
%SJV laag tarief           int64
%Slimme Meter              int64
dtype: object

In [30]:
data.isnull().sum()

JAARTAL                   0
STRAATNAAM                0
POSTCODE_VAN              0
POSTCODE_TOT              0
PRODUCTSOORT              0
Aantal Aansluitingen      0
%Leveringsrichting        0
%Fysieke status           0
%Soort aansluiting        0
Soort aansluiting Naam    0
SJV                       0
%SJV laag tarief          0
%Slimme Meter             0
dtype: int64

In [31]:
data.head()

,JAARTAL,STRAATNAAM,POSTCODE_VAN,POSTCODE_TOT,PRODUCTSOORT,Aantal Aansluitingen,%Leveringsrichting,%Fysieke status,%Soort aansluiting,Soort aansluiting Naam,SJV,%SJV laag tarief,%Slimme Meter
0,2018,De Ruyterkade Steigers,1011AA,1011AB,0,35,10,3,31,3x25,14403,4,2
1,2018,De Ruyterkade,1011AC,1011AC,0,39,10,5,28,3x80,18739,5,2
2,2018,De Ruyterkade,1011AC,1011AC,1,24,10,7,38,G6,6589,0,2
3,2018,Oosterdokskade,1011AD,1011AD,0,19,10,0,58,3x25,4026,5,0
4,2018,Oosterdokskade,1011AD,1011AE,1,11,10,0,82,G4,710,0,0


In [55]:
#multivariate nongraphical data
groups1 = data.groupby(['%Leveringsrichting', 'PRODUCTSOORT'])
grouped1 = groups1['SJV'].agg(np.mean)
grouped1 = grouped1.to_frame().reset_index()
grouped1.columns = ['%Leveringsrichting','PRODUCTSOORT', 'SJV']
grouped1.drop(['PRODUCTSOORT'], axis = 1, inplace = True)
grouped1

,%Leveringsrichting,SJV
0,0,3430.000000
1,1,2040.230769
2,2,2292.666667
3,3,3774.941176
4,4,2973.888889
5,5,3143.540541
6,6,4069.205128
7,7,3741.951076
8,8,4008.451172
9,9,3859.021019


In [41]:
# multivariate graphical data
# plotted mean sjv against %leveringsrichting bins
bin_list = ['0','1','2','3','4','5','6','7','8','9','10']
sjv_list = [x for x in grouped1['SJV']]
dic_data = {'bins': bin_list,
            'sjv' : sjv_list[:11]}

source = ColumnDataSource(data=dic_data)

p1 = figure(x_range=bin_list, y_range=(2000, 5000), plot_height=500, title="Gemiddeld verbruik per leveringsrichtinggroep",
           toolbar_location=None, tools=["pan","zoom_in","wheel_zoom","zoom_out"], x_axis_label='leveringsrichtinggroepen',
          y_axis_label='sjv')



p1.vbar(x=dodge('bins', 0, range=p1.x_range), top='sjv', width=0.2, source=source,
       color="#718dbf")


p1.x_range.range_padding = 0.1
p1.xgrid.grid_line_color = None




In [34]:
#multivariate nongraphical data
groups2 = data.groupby(['JAARTAL','PRODUCTSOORT'])
grouped2 = groups2['SJV'].agg(np.mean)
grouped2 = grouped2.to_frame().reset_index()
grouped2.columns = ['JAARTAL','PRODUCTSOORT','SJV']
grouped2

,JAARTAL,PRODUCTSOORT,SJV
0,2014,0,3619.182824
1,2014,1,1375.367885
2,2015,0,3489.303779
3,2015,1,1334.835799
4,2016,0,3458.330525
5,2016,1,1309.590389
6,2017,0,3422.379748
7,2017,1,1300.376403
8,2018,0,3387.158520
9,2018,1,1299.981850


In [35]:
# mulitvariate graphical data
# plotted mean gas/electricity use through the years 
year_list = ['2014','2015','2016','2017','2018']
sjv_list = [x for x in grouped2['SJV']]
dic_data = {'years': year_list,
             'elk' : sjv_list[0::2],
            'gas': sjv_list[1::2]}

source = ColumnDataSource(data=dic_data)

p2 = figure(x_range=year_list, y_range=(0, 4000), plot_height=500, title="Gemiddeld verbruik per jaar",
           toolbar_location=None, tools=["pan","zoom_in","wheel_zoom","zoom_out"])

p2.vbar(x=dodge('years', -0.125, range=p2.x_range), top='elk', width=0.2, source=source,
       color="#c9d9d3", legend=value("elektriciteit"))

p2.vbar(x=dodge('years',  0.125,  range=p2.x_range), top='gas', width=0.2, source=source,
       color="#718dbf", legend=value("gas"))

p2.x_range.range_padding = 0.1
p2.xgrid.grid_line_color = None
p2.legend.location = "top_right"
p2.legend.orientation = "horizontal"



In [44]:
# univariate graphical data
# frequency of gas and electricity in dataset

i = 0
j = 0
for x in data['PRODUCTSOORT']:
        if x == 0:
            i = i+1
        if x == 1:
            j = j+1

source = ColumnDataSource(dict(x=['Gas','Elektriciteit'],y=[j,i]))

x_label = ""
y_label = "Aantal aansluitingen"
title = "Hoeveelheid aansluitingen gas en elektriciteit"
p3 = figure(plot_width=600, plot_height=300, tools="save",
        x_axis_label = x_label,
        y_axis_label = y_label,
        title=title,
        x_minor_ticks=2,
        x_range = source.data["x"],
        y_range= ranges.Range1d(start=0,end=100000))
p3.left[0].formatter.use_scientific = False
p3.toolbar.tools = [PanTool(), ResetTool(), WheelZoomTool(), BoxSelectTool()]
p3.toolbar_location='right'




labels = LabelSet(x='x', y='y', text='y', level='glyph',
        x_offset=-13.5, y_offset=0, source=source, render_mode='canvas')

p3.vbar(source=source,x='x',top='y',bottom=0,width=0.3,color='red')

p3.add_layout(labels)


In [45]:
output_file('histograms.html', title="graphs.py")
show(gridplot(p1,p2,p3, ncols=2, plot_width=400, plot_height=400, toolbar_location=None))